# Code to migrate the data files in the archive to JSON

We want all the data in a modern format.

# WWII Data

- allied.xml - list of allied planes
- axis.xml - lise of axis planes
- holtgrewe.xml - This is the primary data file.
- index.xml - Looks like some kind of mapping file???

# Holtgrewe Models

The main location for data (in the WWII collection) is holtgrewe.xml.  It
consists of a series of <model> tags.  Here is an example:

```xml
<model index="1" holtgrewe="001" affiliation="1" cos="Great Britain" com="Great Britain" logo="grb">
  <img name="001"></img>
  <title><![CDATA[Miles M.9A Master Mk I]]></title>
  <type type="Advanced Trainer"></type>
  <firstYR>1941</firstYR>
  <displayedAs>
    <![CDATA[This model represents an aircraft of the Royal Air Force No. ...]]>
  </displayedAs>
  <history>
    <![CDATA[As monoplane fighters began to equip Royal Air Force units, ...]]>
  </history>
  <tech>
    <powerplant>
      <![CDATA[One Rolls-Royce Kestrel XXX, liquid-cooled inline piston engine of 715 hp (533 kW)]]>
    </powerplant>
    <performance>
      <maxspeed>226 mph at 15,000 feet (364 km/h at 4,570 m)</maxspeed>
      <ceiling>28,000 feet (8,500 m)</ceiling>
      <range>500 miles (805 km)</range>
    </performance>
    <weight>4,160 pounds (1,886 kg)</weight>
    <maxweight>5,352 pounds (2,428 kg)</maxweight>
    <dimensions span="38 feet 10 inches (11.85 m)" lengths="30 feet 5 inches (9.27 m)" height="9 feet 3 inches (2.82 m)">
    </dimensions>
    <armament><![CDATA[Provisions for one fixed, forward-firing Vickers 0.303 i...]]></armament>
    <crew num="2 [one student, one instructor]"></crew>
    <comments>
      <![CDATA[Model Builder's Comments: This Master has the typical... "]]>
    </comments>
  </tech>
</model>
```

We desired to convert these models into JSON format for use in a web application.
This could be encoded as follows (editing some fields for clarity):

```json
{
  modelNumber: "001",
  countryOfService: "Great Britain",
  countryOfManufacture: "Great Britain",
  logo: "grb",
  title: "Miles M.9A Master Mk I",
  type: "Advanced Trainer",
  firstYear: 1941,
  displayedAs: "This model represents an aircraft of the Royal Air Force No. ... ",
  history: "As monoplane fighters began to equip Royal Air Force units, ...",
  tech: {
    powerplant: "One Rolls-Royce Kestrel XXX, liquid-cooled inline piston engine of 715 hp (533 kW)",
    performance: {
      maxSpeed: "226 mph at 15,000 feet (364 km/h at 4,570 m)",
      ceiling: "28,000 feet (8,500 m)"
      range: "500 miles (805 km)"
    },
    weight: "4,160 pounds (1,886 kg)",
    maxWeight: "5,352 pounds (2,428 kg)",
    dimensions: {
      span: "38 feet 10 inches (11.85 m)",
      length: "30 feet 5 inches (9.27 m)",
      height: "9 feet 3 inches (2.82 m)"
    }
    armament: "Provisions for one fixed, forward-firing Vickers 0.303 i... ",
    crewSize: "2 [one student, one instructor]",
    comments: "Model Builder's Comments: This Master has the typical... "
  }
}
```

We could make this data more generally useful, by removing formatting from
data fields, allowing the display software to convert units in a standard
way.  As it appears that the standard units are English, I would adopt that
for data values (feet (decimal), miles, pounds, mph).  This would further
convert this model to:

```json
{
  modelNumber: "001",
  countryOfService: "Great Britain",
  countryOfManufacture: "Great Britain",
  logo: "grb",
  title: "Miles M.9A Master Mk I",
  type: "Advanced Trainer",
  firstYear: 1941,
  displayedAs: "This model represents an aircraft of the Royal Air Force No. ... ",
  history: "As monoplane fighters began to equip Royal Air Force units, ...",
  tech: {
    powerplant: {
      engines: 1,
      hp: 715,
      description: "Rolls-Royce Kestrel XXX, liquid-cooled inline piston engine",
    },
    performance: {
      maxSpeed: {
        speed: 226,
        altitude: 15000
       }
      ceiling: 28000,
      range: 500,
    },
    weight: 4160,
    maxWeight: 5352,
    dimensions: {
      span: 38.83,
      length: 30.42,
      height: 9.25,
    }
    armament: "Provisions for one fixed, forward-firing Vickers 0.303 i... ",
    crew: {
      size: 2,
      makeup: "one student, one instructor"
    }
  },
  comments: "Model Builder's Comments: This Master has the typical... "
}
```

In [34]:
!ls -al ../archive/wwii/data

total 1156
drwxrwxr-x 2 mckoss mckoss    4096 Jan 26 16:00 .
drwxrwxr-x 4 mckoss mckoss    4096 Jan 26 17:18 ..
-rw-rw-r-- 1 mckoss mckoss   14879 Jan 26 15:59 allied.xml
-rw-rw-r-- 1 mckoss mckoss   13466 Jan 26 15:59 axis.xml
-rw-rw-r-- 1 mckoss mckoss 1133333 Jan 26 15:59 holtgrewe.xml
-rw-rw-r-- 1 mckoss mckoss    6213 Jan 26 15:59 index.xml


In [ ]:
!pip install xmltodict

In [2]:
DATA_DIR = '../archive/wwii/data'

In [25]:
import json
import xmltodict

with open(f"{DATA_DIR}/holtgrewe.xml", 'r') as f:
    data = f.readlines()

# Parser needs a single top-level element - so we'll wrap the data in a root element
data.insert(1, '<root>\n')
data.append('</root>\n')

models = xmltodict.parse(''.join(data))['root']['model']
print(json.dumps(models[0], indent=2))

{
  "@index": "1",
  "@holtgrewe": "001",
  "@affiliation": "1",
  "@cos": "Great Britain",
  "@com": "Great Britain",
  "@logo": "grb",
  "img": {
    "@name": "001"
  },
  "title": "Miles M.9A Master Mk I",
  "type": {
    "@type": "Advanced Trainer"
  },
  "firstYR": "1941",
  "displayedAs": "This model represents an aircraft of the Royal Air Force No. 8 Flying Training School, Ternhill, England, April 1940.",
  "history": "As monoplane fighters began to equip Royal Air Force units, it became obvious that an advanced trainer was needed to transition new young pilots from their basic elementary trainers to the high performance fighters they would be flying. To fulfill this need, the Air Ministry issued a specification for a high performance trainer. George Miles designed the \"Master\" in response. The new trainer was of smooth aeronautical design and was built of wood. Miles' first prototype, which first flew on June 3, 1937, was rejected by the Air Ministry, but Miles continued on 

In [150]:
import re

current_model = None
errors = []

def reshape_model(m):
  return {
    'modelNumber': parse_model(m['@holtgrewe']),
    'countryOfService': m['@cos'],
    'countryOfManufacture': m['@com'],
    'logo': m['@logo'],
    'title': m['title'],
    'type': m['type']['@type'],
    'firstYear': m['firstYR'],
    'displayedAs': m['displayedAs'],
    'history': m['history'],
    'tech': {
      'powerplant': parse_powerplant(m['tech']['powerplant']),
      'performance': {
        'maxSpeed': parse_speed(m['tech']['performance']['maxspeed']),
        'ceiling': parse_ceiling(m['tech']['performance']['ceiling']),
        'range': parse_range(m['tech']['performance']['range']),
      },
      'weight': parse_weight(m['tech']['weight']),
      'maxWeight': parse_weight(m['tech']['maxweight']),
      'dimensions': {
        'span': parse_dimension(m['tech']['dimensions']['@span']),
        'length': parse_dimension(m['tech']['dimensions']['@lengths']),
        'height': parse_dimension(m['tech']['dimensions']['@height']),
      },
      'armament': m['tech']['armament'],
      'crew': parse_crew(m['tech']['crew']['@num']),
    },
    'comments': m['tech']['comments'],
  }

def parse_model(m):
  global current_model
  current_model = m
  return m

def parse_powerplant(d):
  engines = 1
  if re.match(r'One ', d):
    d = d[4:]
  elif re.match(r'Two ', d):
    d = d[4:]
    engines = 2

  result = {
    'engines': engines,
  }

  m = re.match(r'(.*) of ([\d,]+) hp (.*)', d)

  if m is None:
    error(f"Couldn't parse powerplant: {d}")
  else:
    result['hp'] = parse_int(m.group(2))
    d = m.group(1)

  result['description'] = d
  
  return result

def parse_speed(s):
  m = re.match(r'(\d+) mph at ([\d,]+) feet', s)
  if m is None:
    error(f"Couldn't parse speed: {s}")
    return None

  return {
    'speed': int(m.group(1)),
    'altitude': parse_int(m.group(2))
  }

def parse_ceiling(c):
  m = re.match(r'([\d,]+) feet', c)

  if m is None:
    error(f"Couldn't parse ceiling: {c}")
    return None

  return parse_int(m.group(1))

def parse_range(r):
  m = re.match(r'(\d+) miles', r)
  if m is None:
    error(f"Couldn't parse range: {r}")
    return None

  return parse_int(m.group(1))

def parse_weight(w):
  if w is None:
    error("Missing weight");
    return None

  m = re.match(r'([\d,]+) pounds', w)
  if m is None:
    error(f"Couldn't parse weight: {w}")
    return None

  return parse_int(m.group(1))

def parse_dimension(d):
  m = re.match(r'(\d+) feet ([\d]+) inches', d)
  if m is None:
    error(f"Couldn't parse dimension: {d}")
    return None

  ft = parse_int(m.group(1))
  inch = parse_int(m.group(2))

  return round(ft + inch/12, 2)

def parse_crew(c):
  m = re.match(r'(\d+|None|Two),? *(.*)?', c)
  if m is None:
    error(f"Couldn't parse crew: {c}")
    return None

  if m.group(1) == 'None':
    size = 0
  elif m.group(1) == 'Two':
    size = 2
  else:
    size = parse_int(m.group(1))

  result = {
    'size': size
  }

  if m.group(2) is not None and m.group(2) != '':
    m_d = re.match(r'\[(.+)\]', m.group(2))
    if m_d is not None:
      result['makeup'] = m_d.group(1)
    else:
      print(m.group(2))
      result['makeup'] = m.group(2)

  return result

def parse_int(s):
  return int(s.replace(',', ''))

def error(s):
  errors.append(f"{current_model}: {s}")


# Test code on a sample model

print(json.dumps(reshape_model(models[0]), indent=2))

if len(errors) > 0:
  error_lines = '\n'.join(errors)
  print(f"Errors:\n{error_lines}")


{
  "modelNumber": "001",
  "countryOfService": "Great Britain",
  "countryOfManufacture": "Great Britain",
  "logo": "grb",
  "title": "Miles M.9A Master Mk I",
  "type": "Advanced Trainer",
  "firstYear": "1941",
  "displayedAs": "This model represents an aircraft of the Royal Air Force No. 8 Flying Training School, Ternhill, England, April 1940.",
  "history": "As monoplane fighters began to equip Royal Air Force units, it became obvious that an advanced trainer was needed to transition new young pilots from their basic elementary trainers to the high performance fighters they would be flying. To fulfill this need, the Air Ministry issued a specification for a high performance trainer. George Miles designed the \"Master\" in response. The new trainer was of smooth aeronautical design and was built of wood. Miles' first prototype, which first flew on June 3, 1937, was rejected by the Air Ministry, but Miles continued on with the project as a private venture. With the need for an adva

In [151]:
for m in models:
  out = reshape_model(m)
  # print(json.dumps(out['tech']['crew'], indent=2))

print(f"Finished parsing with model '{current_model}'.  {len(errors)} errors.")
print('\n'.join((err for err in errors if 'crew' in err)))
print('\n'.join(errors))

or 3
or 3
or 5
or 5
to 6
to 7, usually 7
to 6 dependent upon training role
plus 3 or 4 passengers
with 3 passengers
or 3
or 7
to 5
to 6
to 9
to 7
or 5 [excluding passengers]
in fighter, none in converted bomber
to 8
or 3
to 5 dependent upon training role
or 5
crew, seating for 8 passengers
to 6
or 8
to 6
to 5
or 2
to 6
or 4
or 3
to 7 [dependent on function]
or 5
to 5
or 6
and up to 3 passengers
or 4
or 5
pilots, maximum 25 troops
to 5 [dependent upon training role]
to 12
to 9
to 10
or 7 [excluding passengers]
to 5
or 5
to 8
with up to 8 passengers
to 6, including students
or 7
observer
to 10
or 2
or 2 [4 to 6 passengers]
to 6, dependent upon role
if no bomb load, up to 24 fully equipped troops
Finished parsing with model '426'.  486 errors.
305: Couldn't parse crew: Usually crew of 2 and up to 6 passengers
320: Couldn't parse crew: Flight crew of 5 or 6
004: Couldn't parse range: 1,055 miles (1,700 km)
007: Couldn't parse dimension: 27 feet 1 inch (8.27 m)
009: Couldn't parse range: 1,